In [1]:
# bow vs tfidf

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub
import warnings
warnings.filterwarnings('ignore')

In [2]:
mlflow.set_tracking_uri('https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow')
dagshub.init(repo_owner='Pravat-21', repo_name='MLops-Mini-Project', mlflow=True)

Accessing as Pravat-21

Initialized MLflow to track repo "Pravat-21/MLops-Mini-Project"

Repository Pravat-21/MLops-Mini-Project initialized!

In [3]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [5]:
# Normalize the text data
df = normalize_text(df)

In [6]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [8]:
mlflow.set_experiment("Bow vs TfIdf-Experiment-2")

2025/10/21 18:22:25 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf-Experiment-2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c5df64e476ff466d8695f6e830ed5a7a', creation_time=1761051146057, experiment_id='1', last_update_time=1761051146057, lifecycle_stage='active', name='Bow vs TfIdf-Experiment-2', tags={}>

In [9]:
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

In [10]:
# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [11]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments of BoW vs TFIDF") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                #mlflow.log_artifact(__file__)
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2025/10/21 18:24:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
🏃 View run LogisticRegression with BoW at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/0ec4718ba02f49df8614a0b8ed4ffdc8
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1


2025/10/21 18:24:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/3e6b30b833c14e3896ee2597e5dae114
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1


2025/10/21 18:24:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Algorithm: RandomForest, Feature Engineering: TF-IDF
Accuracy: 0.7701204819277109
Precision: 0.7642436149312377
Recall: 0.7665024630541872
F1 Score: 0.765371372356124
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/5e974b89eec74d3f965c996fe26e5350
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1


2025/10/21 18:29:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7257831325301205
Precision: 0.7973333333333333
Recall: 0.5891625615763547
F1 Score: 0.6776203966005666
🏃 View run GradientBoosting with BoW at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/b180ce649c174714877f2d7f5e11287d
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1


2025/10/21 18:30:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7214457831325302
Precision: 0.8022130013831259
Recall: 0.5714285714285714
F1 Score: 0.667433831990794
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/ed03b170a3b342388f991cbe341675cd
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1
🏃 View run All Experiments of BoW vs TFIDF at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1/runs/243e7a7e94ea40b1a5a8233c125873e7
🧪 View experiment at: https://dagshub.com/Pravat-21/MLops-Mini-Project.mlflow/#/experiments/1
